In [1]:
import datetime
import gc
import json
import os
import sys
import random

import neptune
import torch
from neptune.utils import stringify_unsupported

from src.attackers import get_attacker
from src.config.meta_config import get_pydantic_models_from_path
from src.evaluator import Evaluator
from src.gradio import run_gradio
from src.server import Server
from src.utils import create_open

/ephemeral/taremu/miniconda3/envs/ws/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfgs = get_pydantic_models_from_path("custom.yaml")
print(f"Number of configs: {len(cfgs)}")

Number of configs: 1


In [3]:
for cfg in cfgs:
    print(cfgs)

[WsConfig(meta=MetaConfig(device='cuda:7', use_neptune=False, neptune_project='ORG/PROJ', seed=123, out_root_dir='out_llama/', result_dir='results/default'), gradio=GradioConfig(skip=True, make_public=False, port=7860, default_prompt='Write a long essay about war.'), server=ServerConfig(model=ModelConfig(skip=False, name='google/gemma-2b-it', use_fp16=True, use_flashattn2=True, prompt_max_len=800, response_max_len=800, n_beams=1, use_sampling=True, sampling_temp=0.7), watermark=WatermarkConfig(scheme=<WatermarkScheme.KGW: 'kgw'>, generation=WatermarkGenerationConfig(seeding_scheme='selfhash', gamma=0.25, delta=4.0), detection=WatermarkDetectionConfig(normalizers=[], ignore_repeated_ngrams=True, z_threshold=4.0)), disable_watermark=False), attacker=AttackerConfig(algo=<AttackerAlgo.OUR: 'our'>, model=ModelConfig(skip=False, name='google/gemma-2b-it', use_fp16=True, use_flashattn2=True, prompt_max_len=800, response_max_len=800, n_beams=1, use_sampling=True, sampling_temp=0.7), querying=A

In [4]:
for cfg in cfgs:
    out_dir = cfg.get_result_path()
    print(out_dir)
    with create_open(f"{out_dir}/config.txt", "w") as f:
        json.dump(cfg.model_dump(mode="json"), indent=4, fp=f)
    run = None
    server = Server(cfg.meta, cfg.server)
    # print(server.watermarks[0].spawn_logits_processor().hash_key)
    # print(server.watermarks[0].spawn_logits_processor().rng)
    attacker = get_attacker(cfg)
    attacker.query_server_and_save(server)
    attacker.load_queries_and_learn(base=False)
    attacker.load_queries_and_learn(base=True)
    evaluator = Evaluator(
            cfg.meta.seed,
            cfg.evaluator,
            server,
            verbose=True,
            neptune_project=cfg.meta.neptune_project,
            run=run,
        )
    evaluator.run_eval(server, attacker, out_dir=out_dir)
    server = None  # type: ignore
    attacker = None  # type: ignore
    evaluator = None  # type: ignore
    run = None
    gc.collect()
    torch.cuda.empty_cache()

    print("Done")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7e27f77b5ab0>>
Traceback (most recent call last):
  File "/ephemeral/taremu/miniconda3/envs/ws/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
